In [10]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.externals import joblib

In [11]:
df = pd.read_csv('eventlog_gui.csv')
print(df) 

       eventID        account  \
0         5140            dc$   
1         5140            dc$   
2         5140            dc$   
3         5140            dc$   
4         5140            dc$   
5         5140            dc$   
6         5140        fsadmin   
7         5140        fsadmin   
8         5140        fsadmin   
9         5140        fsadmin   
10        5140            dc$   
11        5140            dc$   
12        5140            dc$   
13        5140            dc$   
14        5140  local service   
15        5140        filesv$   
16        5140        filesv$   
17        5140            dc$   
18        5140            dc$   
19        5140  local service   
20        5140        filesv$   
21        5140        filesv$   
22        5140            dc$   
23        5140            dc$   
24        5140            dc$   
25        5140            dc$   
26        5140            dc$   
27        5140            dc$   
28        5140  local service   
29        

One-class SVM

In [12]:
def learning(eventid, df, nu, gamma):

    global n_correct_train_old
    df = df[df.eventID == eventid]
    data_dummies = pd.get_dummies(df.iloc[:,1:])
    data_dummies = pd.concat([df.iloc[:,0], data_dummies], axis=1)
    data_dummies.to_csv('data_dummies_' + str(eventid) + '.csv')

    data_normal = data_dummies[data_dummies.target_train == 1]
    X_train = data_normal.ix[:, :-1].values
    
    X_all = data_dummies.ix[:, :-1].values
    X_index = data_dummies.ix[:, -1:].values
    
    clf = svm.OneClassSVM(nu=nu, kernel="rbf", gamma=gamma)
    clf.fit(X_train)

    X_pred_train = clf.predict(X_train)
    n_correct_train = X_pred_train[X_pred_train == 1].size
    n_error_train = X_pred_train[X_pred_train == -1].size
    
    print('Event_ID: ' + str(eventid))
    print('svm.OneClassSVM(nu=' + str(nu) + ', kernel="rbf", gamma=' + str(gamma) + ')')
    print('Training Correct: ' + str(n_correct_train))
    print('Training Error: ' + str(n_error_train))
    print('N: ' + str(n_correct_train+n_error_train))
    print('')

    # Get the best hyper parameter
    if n_correct_train > n_correct_train_old:
        X_train_result = np.concatenate((df[df['target'] == 'train'], X_pred_train[np.newaxis, :].T), axis=1)

        with open('X_train_result' + str(eventid) + '.csv', 'w') as f:
            writer = csv.writer(f)
            writer.writerows(X_train_result)

        joblib.dump(clf, 'ocsvm_gt_' + str(eventid) + '.pkl') 
        n_correct_train_old = n_correct_train


In [13]:
nu_list = [0.1, 0.01, 0.001]
gamma_list = [0.1, 0.01, 0.001]
n_correct_train_old = 0
for nu in nu_list:
    for gamma in gamma_list:
        learning(4674, df, nu, gamma)

nu_list = [0.1, 0.01, 0.001]
gamma_list = [0.1, 0.01, 0.001]
n_correct_train_old = 0
for nu in nu_list:
    for gamma in gamma_list:
        learning(4688, df, nu, gamma)

Event_ID: 4674
svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
Training Correct: 377
Training Error: 96
N: 473

Event_ID: 4674
svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.01)
Training Correct: 380
Training Error: 93
N: 473

Event_ID: 4674
svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.001)
Training Correct: 384
Training Error: 89
N: 473

Event_ID: 4674
svm.OneClassSVM(nu=0.01, kernel="rbf", gamma=0.1)
Training Correct: 104
Training Error: 369
N: 473

Event_ID: 4674
svm.OneClassSVM(nu=0.01, kernel="rbf", gamma=0.01)
Training Correct: 81
Training Error: 392
N: 473

Event_ID: 4674
svm.OneClassSVM(nu=0.01, kernel="rbf", gamma=0.001)
Training Correct: 107
Training Error: 366
N: 473

Event_ID: 4674
svm.OneClassSVM(nu=0.001, kernel="rbf", gamma=0.1)
Training Correct: 401
Training Error: 72
N: 473

Event_ID: 4674
svm.OneClassSVM(nu=0.001, kernel="rbf", gamma=0.01)
Training Correct: 368
Training Error: 105
N: 473

Event_ID: 4674
svm.OneClassSVM(nu=0.001, kernel="rbf", gamma=0.001)
Training Co